In [ ]:
import psycopg2 as db
from uuid import uuid4
import hashlib
import importnb

with importnb.Notebook():
    import rag

def get_connection():
    conn = db.connect(
        host="localhost",
        database="metadata",
        user="postgres",
        password="khanzode"
    )
    print("Connection established")
    return conn

conn = get_connection()
cur = conn.cursor()

In [ ]:
def init_db():
    cur.execute("""
    CREATE TABLE IF NOT EXISTS docs (
        doc_id SERIAL PRIMARY KEY,
        doc_hash TEXT,
        doc_name TEXT UNIQUE,
        doc_chunk_ids TEXT[],
        status TEXT
    )
    """)
    conn.commit() 
    print("Docs table initialized successfully")

In [ ]:
def get_file_hash(file):
    content = file.read()
    file_hash = hashlib.sha256(content).hexdigest()
    file.seek(0)
    return file_hash
    
def is_uploaded(file):
    print("-"*100)
    file_hash = get_file_hash(file)
    print(f"file hash = {file_hash}")
    try:
        cur.execute("SELECT doc_hash, doc_name, doc_chunk_ids, status FROM docs WHERE doc_name = %s", (file.name,))
        returned_entry = cur.fetchone()
        print(f"returned entry = {returned_entry}")
    except Exception as e:
        print(f"Database error in is_uploaded: {e}")
        import traceback
        traceback.print_exc()
        return 0

    if returned_entry:
        doc_hash, doc_name, doc_chunk_ids, status = returned_entry

        if file_hash == doc_hash:   # same file same content uploaded again
            if status == "Completed":
                return 1    # 1 means that the file is the same, no need to chunk
            elif status == "Pending Vector" and (not doc_chunk_ids or len(doc_chunk_ids) == 0):
                # Allow re-chunking if stuck in Pending Vector with empty chunk_ids
                print("File stuck in Pending Vector with empty chunk_ids, allowing re-chunking.")
                return 0
            else:
                # Other non-completed statuses, allow re-chunking
                print(f"File in status {status}, allowing re-chunking.")
                return 0
        else:   # same file updated case
            cur.execute("UPDATE docs SET doc_hash = %s, doc_chunk_ids = %s, status = %s WHERE doc_name = %s", (file_hash, [], "Pending Vector", file.name))
            print("file updated")
    else:   # document inserted - add entry along with status as "Pending vector"
        cur.execute("INSERT INTO docs (doc_hash, doc_name, doc_chunk_ids, status) VALUES (%s, %s, %s, %s)",(file_hash, file.name, [], "Pending Vector"))
        print("file added for the first time")

    print("-"*100)
    conn.commit()
    return 0    # 0 means that the file is different or needs to be re-chunked. metadata db is updated. now get chunks and store them

def update_insert(chunk_ids, file):
    print("-"*100)
    # Once chunks are completed-add chunk ids against the document
    # once entire doument is chunked and vectorisation is completed mark "completed"
    cur.execute("UPDATE docs SET doc_chunk_ids = %s, status = %s WHERE doc_name = %s",(chunk_ids, "Completed", file.name))
    print("update insert done, should show confirmed completed now")
    conn.commit()
    print("-"*100)

In [ ]:
def mark_for_deletion(file_hash):
    print("-"*100)
    try:
        # Get all chunk Ids and delete those from vector
        cur.execute("SELECT doc_chunk_ids, status from docs WHERE doc_hash = %s", (file_hash,))
        entry = cur.fetchone()
        
        if entry is None:
            print(f"No document found with hash: {file_hash}")
            print("-"*100)
            return []
        
        chunk_ids, status = entry
        print(f"Found document with status: {status}, chunk_ids: {chunk_ids}")
        
        if status == "Completed":   # Only allow deletion if status is "Completed"
            print("Attempting to delete chunks from vector database...")
            
            if chunk_ids:  # Only delete if chunk_ids exist
                try:
                    rag.delete_chunks_from_vectordb(chunk_ids)
                    print("Successfully deleted chunks from vector database")
                except Exception as vector_error:
                    print(f"Error deleting from vector database: {vector_error}")
            else:
                print("No chunk_ids to delete from vector database")
            
            cur.execute("UPDATE docs SET status = %s WHERE doc_hash = %s", ("Deleted Pending Vector", file_hash))
            conn.commit()
            print(f"File with hash {file_hash} marked as 'Deleted Pending Vector' in metadata database")
            
            print("-"*100)
            return chunk_ids if chunk_ids else []
            
        else:
            print(f"Cannot delete document with status: {status}. Only 'Completed' documents can be deleted.")
            print("-"*100)
            return []
            
    except Exception as e:
        print(f"Error in mark_for_deletion: {e}")
        import traceback
        traceback.print_exc()
        conn.rollback()  # Rollback the failed transaction
        print("-"*100)
        return []

In [ ]:
def clear_metadata_db():
    """
    Clear all entries from the metadata database.
    This should be called when flush_db() is used to clear the vector database.
    """
    print("-"*100)
    try:
        cur.execute("DELETE FROM docs")
        conn.commit()
        print("All entries cleared from metadata database")
        
        # Reset the sequence for doc_id if needed
        cur.execute("ALTER SEQUENCE docs_doc_id_seq RESTART WITH 1")
        conn.commit()
        print("Document ID sequence reset")
        
    except Exception as e:
        print(f"Error clearing metadata database: {e}")
        conn.rollback()
    print("-"*100)